# MLP MNIST
Hier werden das erste Mal im MNIST nun Algotihmen aus dem 
Multilayer Perceptron (MLP) for multi-class softmax classification:

In [11]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

In [12]:
# Hier die Hyperparamter
batch_size = 128
num_classes = 10
epochs = 20

In [13]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [14]:
x_train = x_train.reshape(60000, 784) # MLP benötigt InputVektor
x_test = x_test.reshape(10000, 784) # MLP benötigt InputVektor
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 # skalieren
x_test /= 255 # Skalieren
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [15]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [16]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax')) # hier der letzte Layer hat die Anzahl der möglichen klassen & die Softmax Aktivierung
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


In [17]:
#
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])


In [20]:
# Ausführen des Modelles
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1, # show training progress during training Possible = 0,1,2
                    validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
18048/60000 [========>.....................] - ETA: 8s - loss: 0.0756 - accuracy: 0.9757 ETA: 8s - loss: 0.0762 - accuracy: 0.

KeyboardInterrupt: 

In [11]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.14572093418701815
Test accuracy: 0.9824000000953674
